In [1]:
# google colab setup
from google.colab import drive
drive.mount("/content/gdrive")

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!pip install fast-ml
# !CMDSTAN=/tmp/cmdstan-2.22.1 STAN_BACKEND=CMDSTANPY pip install prophet
# !pip install pystan==2.19.1.1
# !pip install prophet

In [ ]:
# imports
import os
from pathlib import Path

from sklearn import ensemble, metrics
import matplotlib.pyplot as plt
from fast_ml import model_development as md
import pandas as pd
import tensorflow as tf
import numpy as np

# from prophet import Prophet
# from prophet.plot import plot_plotly, plot_components_plotly
# from prophet.diagnostics import cross_validation
# from prophet.plot import plot_cross_validation_metric

In [ ]:
# use_tpu = True

# if use_tpu:
#     assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

# if 'COLAB_TPU_ADDR' in os.environ:
#   TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
# else:
#   TF_MASTER=''

# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TF_MASTER)
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
# pathing
project_dir = Path("gdrive/MyDrive/Assistance/Shivam/teamPoseidon")

data_file = project_dir / "final_dataset_csv.csv"

In [ ]:
# Hypers
SPLITS = {
    "train":0.7,
    "validation":0.2,
    "test":0.1,
}

# they definitely needs optimization.
BATCH_SIZE = 64 # bacth size in batch-SGD/variants
BUFFER_SIZE = 10 # for shuffling the dataset
STEP = 1 # for creation of dataset

# Train and evaluate
STEPS_PER_EPOCH = 10 # hyperparameter
EPOCHS = 100 # hyperparameter

FORECAST_LENGTH = 90

HISTORY_LENGTH = 10


In [ ]:
# data loading
df = pd.read_csv(data_file, sep=',', na_values=[" ", "&nbsp;"], parse_dates=["FLOW_DATE"], header=0)
df = df.ffill()
df = df.set_index("FLOW_DATE")
df["dummy"] = 1

In [ ]:

df["RES_LEVEL_FT"]

In [ ]:
# prediction_df = df.copy()
# prediction_df["ds"] = prediction_df.index
# prediction_df['y'] = prediction_df['RES_LEVEL_FT']
# prediction_df["cap"] = prediction_df["RES_LEVEL_FT"].max()

# model = Prophet(
#     growth="flat",
#     seasonality_mode="multiplicative",
#     daily_seasonality=False,
#     weekly_seasonality=False,
#     changepoint_prior_scale=0.01
# )

# model.fit(prediction_df)

# future = model.make_future_dataframe(periods=3*365)
# forecast = model.predict(future)

# forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
# plot_plotly(model, forecast)

In [ ]:
# forecast

In [ ]:
# plot_components_plotly(model, forecast)

In [ ]:
# df_cv = cross_validation(model, initial='730 days', period='365 days', horizon = '90 days')

In [ ]:
# ## Add cross validation
# fig = plot_cross_validation_metric(df_cv, metric='mse')

In [ ]:
used_cols = ['PRESENT_STORAGE_TMC', 'INFLOW_CUSECS', 'OUTFLOW_CUECS', 'tempC', 'windspeedKmph', 'precipMM', 'humidity', 'pressure (mB)', 'cloudcover (%)', 'HeatIndexC', 'DewPointC', 'WindChillC', 'WindGustKmph', 'RES_LEVEL_FT']
target_col = 'RES_LEVEL_FT'

# X_train, y_train, X_valid, y_valid, X_test, y_test = md.train_valid_test_split(
#     df[used_cols + ["dummy"]],
#     target="dummy", # use dummy for train test split before preproccessing
#     train_size=SPLITS["train"],
#     valid_size=SPLITS["validation"],
#     test_size=SPLITS["test"],
#     method="sorted",
#     sort_by_col="FLOW_DATE",
# )

# data_dict = {
#     "train": X_train,
#     "validation": X_valid,
#     "test": X_test,
# }

In [ ]:
# windowed_ys = tf.keras.preprocessing.timeseries_dataset_from_array(
#     data=data_dict["train"][target_col],
#     targets=None,
#     sequence_length=FORECAST_LENGTH,
#     sequence_stride=1,
#     shuffle=False,
#     batch_size=1,
# )

In [ ]:
def get_windowed_dataset(
    dataframe,
    target_col,
    history_length=HISTORY_LENGTH,
    forecast_length=FORECAST_LENGTH,
    batch_size=32,
    stride=30,
    ):

    data = dataframe.values.copy()
    ys = dataframe[target_col]

    data_future = data.copy()
    # print(data_future.shape)
    data = data[:-forecast_length-history_length]
    # print(data.shape)
    ys = ys[forecast_length+history_length:] # predict window starting at next time step

    # window the labels to make forecasts in the future
    windowed_ys = tf.keras.preprocessing.timeseries_dataset_from_array(
        data=ys,
        targets=None, # ys.index,
        sequence_length=forecast_length,
        sequence_stride=stride,
        shuffle=False,
        batch_size=1,
    )
    print(data.shape)
    ys = np.array(list(windowed_ys.as_numpy_iterator())).squeeze()
    dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
        data=data,
        targets=ys,
        sequence_length=history_length,
        sequence_stride=stride,
        shuffle=False,
        batch_size=batch_size,
    )
    print(data_future.shape)
    future_dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
        data=data_future,
        targets=np.ones((data_future.shape[0],1)),
        sequence_length=history_length,
        sequence_stride=stride,
        shuffle=False,
        batch_size=batch_size,
    )
    return dataset, future_dataset

In [ ]:
def dataset_to_arrays(dataset, flatten=True, last_y=True):
    all_xs = []
    all_ys = []
    for x, y in dataset.as_numpy_iterator():
        all_xs.append(x)
        all_ys.append(y)
    ys = np.concatenate(all_ys)
    xs = np.concatenate(all_xs)
    if flatten:
        xs = xs.reshape(xs.shape[0], -1)
    if last_y:
        ys = ys[:, -1]
    return xs, ys

In [ ]:
historical_dataset, future_dataset  = get_windowed_dataset(
    df[used_cols],# + ["dummy"]],
    target_col=target_col,
    stride=1
    )

historical_xs, historical_ys = dataset_to_arrays(historical_dataset)
future_xs, _ = dataset_to_arrays(future_dataset)
print(historical_xs.shape, historical_ys.shape, future_xs.shape)

data_df = pd.DataFrame(np.concatenate([np.arange(historical_ys.shape[0])[:, None], historical_xs, historical_ys[:,None]], axis=-1))

X_train, y_train, X_valid, y_valid, X_test, y_test = md.train_valid_test_split(
    data_df,
    target=data_df.columns[-1], # use dummy for train test split before preproccessing
    train_size=SPLITS["train"],
    valid_size=SPLITS["validation"],
    test_size=SPLITS["test"],
    method="sorted",
    sort_by_col=data_df.columns[0],
)

# use .iloc[:,1:]to drop first column
data_dict = {
    "train": {"X":X_train.iloc[:,1:], "y":y_train},
    "validation": {"X":X_valid.iloc[:,1:], "y":y_valid},
    "test": {"X":X_test.iloc[:,1:], "y":y_test},
}

X_means = data_dict["train"]["X"].mean()
X_stdev = data_dict["train"]["X"].std()
y_means = data_dict["train"]["y"].mean()
y_stdev = data_dict["train"]["y"].std()
for partition in data_dict.keys():
    data_dict[partition]["X"] = (data_dict[partition]["X"] - X_means) / X_stdev
    data_dict[partition]["y"] = (data_dict[partition]["y"] - y_means) / y_stdev

future_xs = (future_xs - X_means[None,]) / X_stdev[None,:]
# future_xs = (future_xs - X_means[None, :140]) / X_stdev[None, :140]

In [ ]:
# dataset_dict = {
#     partition_name: get_windowed_dataset(partition_data, target_col=target_col)
#     for partition_name, partition_data in data_dict.items()
# }
# future_dataset_dict = {k:v[1] for k,v in dataset_dict.items()}
# dataset_dict = {k:v[0] for k,v in dataset_dict.items()}

In [ ]:
# x_example,y_example = next(dataset_dict["train"].take(1).as_numpy_iterator())
# print("X shape", x_example.shape)
# print("y shape", y_example.shape)

In [ ]:
# lstm training
# with strategy.scope():

#     multi_step_model = tf.keras.models.Sequential()
#     # multi_step_model.add(tf.keras.layers.Conv1D(
#     #     filters=32,
#     #     kernel_size=30,
#     #     input_shape=x_example.shape[-2:],
#     #     ))
#     # # multi_step_model.add(tf.keras.layers.BatchNormalization())
#     # multi_step_model.add(tf.keras.layers.LeakyReLU())

#     # multi_step_model.add(tf.keras.layers.BatchNormalization())
#     # multi_step_model.add(tf.keras.layers.Conv1D(
#     #     filters=32,
#     #     kernel_size=30,
#     #     )
#     # )
#     # multi_step_model.add(tf.keras.layers.LeakyReLU())

#     # multi_step_model.add(tf.keras.layers.BatchNormalization())
#     # multi_step_model.add(tf.keras.layers.Conv1D(
#     #     filters=32,
#     #     kernel_size=30,
#     #     )
#     # )
#     # multi_step_model.add(tf.keras.layers.LeakyReLU())
#     # multi_step_model.add(tf.keras.layers.Flatten())
#     multi_step_model.add(tf.keras.layers.LSTM(
#         32,
#         return_sequences=True,
#         input_shape=x_example.shape[-2:],
#         # activation="relu",
#         ),
#     )
#     multi_step_model.add(tf.keras.layers.LSTM(
#         8, 
#         activation='relu',
#         )
#     )
#     multi_step_model.add(tf.keras.layers.Dense(90)) # for 90 outputs
#     # multi_step_model.add(tf.keras.layers.LeakyReLU(alpha=0.05))
#     multi_step_model.add(tf.keras.layers.Dense(FORECAST_LENGTH)) # for 90 outputs

#     # multi_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), loss='mae')
#     # multi_step_model.compile(optimizer=tf.keras.optimizers.Adam(clipvalue=1.0), loss='mae')
#     multi_step_model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=0.1, clipvalue=1.0),
#         loss='mae'
#         )
# early_stopping_cb = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss',
#     patience=5,
#     restore_best_weights=True
#     )

# multi_step_history = multi_step_model.fit(
#     dataset_dict["train"].cache().shuffle(BUFFER_SIZE).repeat(),
#     epochs=EPOCHS,
#     steps_per_epoch=STEPS_PER_EPOCH,
#     validation_data=dataset_dict["validation"],
#     validation_steps=1,
#     callbacks=[early_stopping_cb],
#     )

In [ ]:
# # Plot train and validation loss over epochs
# def plot_train_history(history, title):
#     loss = history.history['loss']
#     val_loss = history.history['val_loss']
#     epochs = range(len(loss))
    
#     plt.figure()

#     plt.plot(epochs, loss, 'b', label='Training loss')
#     plt.plot(epochs, val_loss, 'r', label='Validation loss')
#     plt.title(title)
#     plt.legend()
#     plt.grid()

#     plt.show()

In [ ]:
# plot_train_history(multi_step_history, "lstm")

In [ ]:
def create_time_steps(length):
  return list(range(-length, 0))

#plotting function
def multi_step_plot(history, true_future, prediction, title=""):
  plt.figure(figsize=(12, 6))
  num_in = create_time_steps(len(history))
  num_out = len(true_future)
  plt.grid()
  plt.plot(num_in, np.array(history), label='History')
  plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'bo',
           label='True Future')
  if prediction.any():
    plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',
             label='Predicted Future')
  plt.legend(loc='upper left')
  plt.title(title)
  plt.show()
  


In [ ]:
# partition_name = "train"
# y_pred = multi_step_model.predict_generator(dataset_dict[partition_name])

# all_xs = []
# all_ys = []
# for x,y in dataset_dict[partition_name].as_numpy_iterator():
#     all_xs.append(x)
#     all_ys.append(y)
# y_true = np.concatenate(all_ys)
# xs = np.concatenate(all_xs)

# print("shapes", xs.shape, y_true.shape, y_pred.shape)

# for i in range(0, xs.shape[0],xs.shape[0]//2):
#     multi_step_plot(xs[i, :,-1], y_true[i], y_pred[i], title=f"{partition_name} offset {i}")

In [ ]:
# partition_name = "validation"
# y_pred = multi_step_model.predict_generator(dataset_dict[partition_name])

# all_xs = []
# all_ys = []
# for x,y in dataset_dict[partition_name].as_numpy_iterator():
#     all_xs.append(x)
#     all_ys.append(y)
# y_true = np.concatenate(all_ys)
# xs = np.concatenate(all_xs)

# print("shapes", xs.shape, y_true.shape, y_pred.shape)

# for i in range(0, xs.shape[0],xs.shape[0]//2):
#     multi_step_plot(xs[i, :,-1], y_true[i], y_pred[i], title=f"{partition_name} offset {i}")

In [ ]:
# partition_name = "test"
# y_pred = multi_step_model.predict_generator(dataset_dict[partition_name])

# all_xs = []
# all_ys = []
# for x,y in dataset_dict[partition_name].as_numpy_iterator():
#     all_xs.append(x)
#     all_ys.append(y)
# y_true = np.concatenate(all_ys)
# xs = np.concatenate(all_xs)

# print("shapes", xs.shape, y_true.shape, y_pred.shape)

# for i in range(0, xs.shape[0],xs.shape[0]//2):
#     multi_step_plot(xs[i, :,-1], y_true[i], y_pred[i], title=f"{partition_name} offset {i}")

In [ ]:
# sklearn_datasets_dict = {
#     k:dataset_to_arrays(v)
#     for k,v in dataset_dict.items()
# }

In [ ]:
# for k, (xs, ys) in sklearn_datasets_dict.items():
#     print(f"{k}:{xs.shape}, {ys.shape}")

In [ ]:
# model = ensemble.BaggingRegressor(n_estimators=50)
# model.fit(sklearn_datasets_dict["train"][0], sklearn_datasets_dict["train"][1])

kwargs = dict(
    max_depth=5,
    subsample=0.5,
)

# Each model has to be separate
# lower_model = ensemble.GradientBoostingRegressor(loss="quantile",alpha=0.1,**kwargs)
# mid_model = ensemble.GradientBoostingRegressor(loss="ls", **kwargs)
# upper_model = ensemble.GradientBoostingRegressor(loss="quantile", alpha=0.9, **kwargs)

lower_model = ensemble.GradientBoostingRegressor(loss="quantile",alpha=0.1,**kwargs,)
mid_model = ensemble.GradientBoostingRegressor(loss="ls", **kwargs)
upper_model = ensemble.GradientBoostingRegressor(loss="quantile", alpha=0.9, **kwargs)

lower_model.fit(data_dict["train"]['X'], data_dict["train"]["y"])
mid_model.fit(data_dict["train"]["X"], data_dict["train"]["y"])
upper_model.fit(data_dict["train"]["X"], data_dict["train"]["y"])


In [ ]:
def rescale_y(y):
    # return y
    return y*y_stdev+y_means

In [ ]:
result_dict = {}
for partition, d in data_dict.items():
    y_pred = mid_model.predict(d["X"])
    print(f"{partition} mae:", metrics.mean_absolute_error(rescale_y(d["y"]), rescale_y(y_pred)))
    # multi_step_plot(xs[:, -1], ys, y_pred, title=f"{partition}")

In [ ]:
partition_name = "train"

xs = data_dict[partition_name]["X"]
ys = rescale_y(data_dict[partition_name]["y"])
ypred_lb = rescale_y(lower_model.predict(xs))
ypred_mid = rescale_y(mid_model.predict(xs))
ypred_ub = rescale_y(upper_model.predict(xs))

num_past = xs.shape[0]

num_future = ypred_lb.shape[0]

x_indices_future = np.arange(num_future)
plt.figure(figsize=(20,10))

# plot future predictions
plt.fill_between(x_indices_future, ypred_lb, ypred_ub, color='b', alpha=0.1)
plt.plot(x_indices_future, ypred_mid, "--")
plt.plot(x_indices_future, ys)

plt.title(partition_name)

In [ ]:
partition_name = "validation"

xs = data_dict[partition_name]["X"]
ys = rescale_y(data_dict[partition_name]["y"])
ypred_lb = rescale_y(lower_model.predict(xs))
ypred_mid = rescale_y(mid_model.predict(xs))
ypred_ub = rescale_y(upper_model.predict(xs))

num_past = xs.shape[0]

num_future = ypred_lb.shape[0]

x_indices_future = np.arange(num_future)
plt.figure(figsize=(20,10))

# plot future predictions
plt.fill_between(x_indices_future, ypred_lb, ypred_ub, color='b', alpha=0.1)
plt.plot(x_indices_future, ypred_mid, "--")
plt.plot(x_indices_future, ys)

plt.title(partition_name)

In [ ]:
partition_name = "test"

xs = data_dict[partition_name]["X"]
ys = rescale_y(data_dict[partition_name]["y"])
ypred_lb = rescale_y(lower_model.predict(xs))
ypred_mid = rescale_y(mid_model.predict(xs))
ypred_ub = rescale_y(upper_model.predict(xs))

num_past = xs.shape[0]

num_future = ypred_lb.shape[0]

x_indices_future = np.arange(num_future)
plt.figure(figsize=(20,10))

# plot future predictions
plt.fill_between(x_indices_future, ypred_lb, ypred_ub, color='b', alpha=0.1)
plt.plot(x_indices_future, ypred_mid, "--")
plt.plot(x_indices_future, ys)

plt.title(partition_name)

In [ ]:
# # xgboost
# train mae: 0.09998867638561532
# validation mae: 0.5098038188379816
# test mae: 0.6213932867880211


In [ ]:

X_means.shape

In [ ]:


ypred_lb = rescale_y(lower_model.predict(xs))
ypred_mid = rescale_y(mid_model.predict(xs))
ypred_ub = rescale_y(upper_model.predict(xs))

num_past = xs.shape[0]

num_future = ypred_lb.shape[0]

x_indices_future = np.arange(num_future)
plt.figure(figsize=(20,10))

# plot future predictions
plt.fill_between(x_indices_future, ypred_lb, ypred_ub, color='b', alpha=0.1)
plt.plot(x_indices_future, ypred_mid, "--")
plt.plot(x_indices_future, ys)

plt.title(partition_name)

In [ ]:
historical_xs.shape, X_means.shape

In [ ]:
# refit model on full historical dataset and predict on future dataset

historical_training_xs = (historical_xs - X_means[None,]) / X_stdev[None,]
historical_training_ys = (historical_ys - y_means) / y_stdev

lower_model = ensemble.GradientBoostingRegressor(loss="quantile",alpha=0.1,**kwargs,)
mid_model = ensemble.GradientBoostingRegressor(loss="ls", **kwargs)
upper_model = ensemble.GradientBoostingRegressor(loss="quantile", alpha=0.9, **kwargs)

lower_model.fit(historical_training_xs, historical_training_ys)
mid_model.fit(historical_training_xs, historical_training_ys)
upper_model.fit(historical_training_xs, historical_training_ys)


partition_name = "future"

# d = data_dict[partition_name]
xs = future_xs[-90:]

ypred_lb = rescale_y(lower_model.predict(xs))
ypred_mid = rescale_y(mid_model.predict(xs))
ypred_ub = rescale_y(upper_model.predict(xs))

num_past = xs.shape[0]

num_future = ypred_lb.shape[0]

x_indices_future = np.arange(num_future)
plt.figure(figsize=(20,10))

# plot future predictions
plt.fill_between(x_indices_future, ypred_lb, ypred_ub, color='b', alpha=0.1, label = "Predicted Lower/Upper Bound")
plt.plot(x_indices_future, ypred_mid, "--", label = "Predicted Value")
# plt.plot(x_indices_future, ys)

plt.title("Future Forecasts", fontdict = {'fontsize' : 26})
plt.style.use('tableau-colorblind10')
plt.legend(prop={"size":20})
plt.grid(True)
plt.savefig("future_forecast.png", dpi = 1200)

In [ ]:
data_to_dump = {
    "mid":ypred_mid,
    "lb": ypred_lb,
    "ub":ypred_ub,
}

In [ ]:
pd.DataFrame(data_to_dump).to_csv("future_predictions.csv", index=False)

In [ ]:
df.index.shape

In [ ]:
pd.to_timedelta(FORECAST_LENGTH, unit="day")

In [ ]:
feature_names = [f"{c} d-{i}" for i in range(HISTORY_LENGTH-1, -1, -1) for c in used_cols]
feature_importance_df = pd.DataFrame({"feature":feature_names, "importance":mid_model.feature_importances_})

In [ ]:
print(feature_importance_df.sort_values("importance", ascending=False).head(10))